# Project 1: Training a Simple Neural Network with GPU

## Introduction

In this project, you will create, train, and evaluate a simple neural network using both TensorFlow and PyTorch. The objective is to ensure you are comfortable with setting up a neural network and utilizing GPU acceleration for training. 

The code for this assignment will mirror that of Lab 1, but you will need to complete some items, and make some changes.  You will use the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) for this project.

## Objectives

1. Set up TensorFlow and PyTorch environments.
2. Verify GPU availability.
3. Implement a simple neural network in TensorFlow and PyTorch.
4. Train and evaluate the models.
5. Answer assessment questions.

## Instructions

Follow the steps below to complete the project. Ensure that you use a GPU to train your models.

---

### Step 1: Set Up Your Environment

First, install the necessary libraries. Run the following cell to install TensorFlow and PyTorch. 


Provide snapshots from your environment showing:
1) You are using a virtual environment
2) You have installed `TensorFlow` and `PyTorch`

---

### Step 2: Verify GPU Availability
Check if TensorFlow and PyTorch can detect the GPU.

Run the following two code blocks and show the output.

In [21]:
# GPU Info
print("\n=== NVIDIA GPU Info ===")
!nvidia-smi

# # CUDA Version
# print("\n=== CUDA Version ===")
# !nvcc --version

# # cuDNN Version
# print("\n=== cuDNN Version ===")
# !cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

# # CPU Info
# print("\n=== CPU Model ===")
# !lscpu | grep "Model name"

# # RAM Info
# print("\n=== RAM Info ===")
# !free -h | grep Mem

# OS Info
print("\n=== OS Information ===")
!cat /etc/os-release | grep PRETTY_NAME

# Python Version
print("\n=== Python Version ===")
!python3 --version


=== NVIDIA GPU Info ===
Fri Jun  6 22:30:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     Off |   00000000:01:00.0  On |                  N/A |
| N/A   42C    P8              1W /   80W |    4772MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------------------

#### TensorFlow GPU Check

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")

TensorFlow version: 2.15.0
GPU is available for TensorFlow!


#### PyTorch GPU Check

In [3]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
elif torch.mps.is_available():
    print("MPS (Apple Silicon GPU support) is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")

PyTorch version: 2.5.1+cu121
GPU is available for PyTorch!


---

### Step 3: Implement and Train a Simple Neural Network
#### TensorFlow Implementation
1. Load and preprocess the Fashion MNIST dataset.
2. Define the neural network model.  Departing from the example in class, use 2 hidden layers, each with 64 units and relu activation functions, densely connected.
3. Compile the model.
4. Train the model using the GPU.  Use a batch size of 64, and run 6 epochs.
5. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
fashion_mnist_data = tf.keras.datasets.fashion_mnist.load_data()
(x_train, y_train), (x_test, y_test) = fashion_mnist_data
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
# with tf.device('/GPU:0'):
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5340 - accuracy: 0.8119 - val_loss: 0.4156 - val_accuracy: 0.8520
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3896 - accuracy: 0.8582 - val_loss: 0.3841 - val_accuracy: 0.8612
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3524 - accuracy: 0.8706 - val_loss: 0.3604 - val_accuracy: 0.8702
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3297 - accuracy: 0.8776 - val_loss: 0.3503 - val_accuracy: 0.8687
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3823 - accuracy: 0.8624
Test accuracy: 0.8624


#### PyTorch Implementation
1. Load and preprocess the Fashion MNIST dataset.  Set batch size to 64.
2. Define the neural network model.  Departing from the example in class, use 2 hidden layers, each with 64 units and relu activation functions, densely connected.  
4. Define loss function and optimizer.
5. Train the model using the GPU.  Run 6 epochs.
6. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.hidden1 = nn.Linear(28*28, 64)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 64)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(64, 10)  # for 10 classes, e.g., MNIST
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.hidden1(x)
        x = self.relu1(x)
        x = self.hidden2(x)
        x = self.relu2(x)
        x = self.output(x)
        return x

model = SimpleNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check for GPU
if torch.cuda.is_available():
    dev = 'cuda'
elif torch.mps.is_available():
    dev = 'mps'
else:
    dev = 'cpu'
device = torch.device(dev)
model.to(device)


# Train the model
num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Epoch [1/6], Loss: 0.3309
Epoch [2/6], Loss: 0.5422
Epoch [3/6], Loss: 0.3912
Epoch [4/6], Loss: 0.4251
Epoch [5/6], Loss: 0.3989
Epoch [6/6], Loss: 0.3248
Test Accuracy: 86.67%


---
### Questions
Answer the following questions in detail.

1. What is the purpose of normalizing the input data in both TensorFlow and PyTorch implementations?
2. Explain the role of the activation function relu in the neural network.
3. Why is it important to use GPU for training neural networks?
4. Compare the training time and accuracy of the TensorFlow and PyTorch models. Which one performed better and why?


## Answer 1

Normalization is the process of scaling input data so that its values lie within a specific range (commonly `[-1, 1]` or `[0, 1]`).

In both TensorFlow and PyTorch, we normalize FashionMNIST images—originally with pixel values ranging from 0 to 255—by dividing by 255. This scales the values to the range `[0, 1]`.

**Purpose:**
- Faster convergence during training  
- Improved generalization of the model  
- Ensures input features have similar scales, helping the network learn more effectively


## Answer 2

**ReLU** (Rectified Linear Unit) introduces non-linearity into the neural network, allowing it to learn complex patterns.

**Benefits:**
- Enables learning of non-linear relationships  
- Simple and efficient to compute  
- Reduces vanishing gradient issues (positive inputs have gradient = 1)  
- Promotes sparse activation (sets negatives to zero), improving generalization


## Answer 3

**GPU Importance:**
- Enables massive parallel processing for matrix and tensor operations  
- ALOT faster than CPU for large models  
- Saves time and energy during training  
- Makes training large models feasible and scalable

**Note:** For small tasks, CPUs may suffice, but GPUs are essential for complex models or big datasets.


## Answer 4

**Results Summary:**

| Framework  | Training Time | Accuracy |
|------------|---------------|----------|
| TensorFlow | 16.7 sec      | 86.24%   |
| PyTorch    | 59.6 sec      | 86.67%   |

**Which Performed Better & Why:**

- **TensorFlow** trained faster (16.7s vs 59.6s)  
- **PyTorch** had slightly higher accuracy (86.67% vs 86.24%)  
- Speed difference likely due to TensorFlow's better GPU/data pipeline usage  
- Accuracy gap is minor and likely from random variation

**Conclusion:**  
In this Scenario with my Particular Specifications `TensorFlow` is better for speed; `PyTorch` slightly better in accuracy. Both are suitable for this task.

---
### Submission
Submit a link to your completed Jupyter Notebook (e.g., on GitHub (private) or Google Colab) with all the cells executed, and answers to the assessment questions included at the end of the notebook.